# Julia version of bonding models for PES !!

### Basic Optimization tests

In [8]:
using Optim, LsqFit # opt libs
using LinearAlgebra # self explanatory
using ReverseDiff: GradientTape, GradientConfig, gradient, gradient!, compile, DiffResults # autodiff
using DelimitedFiles, MLUtils, BenchmarkTools # utils

In [116]:
# examples of "closures", a technique to fix args!
f(x::Vector, a, b) = (x[1] - a)^2 + (x[2] - b)^2

using Optim
g(x::Vector) = f(x, 3, 4)
optimize(g, [0.,0.])

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [117]:
# another closure example in Julia, anonymous function:
optimize(x -> f(x,3,4), [0.,0.]) 

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [2]:
# functional form:
function f_ratpot_2(Θ, R, M)
    #=
    ansatz 1 for diatomic potential
    params:
        - Θ := training parameters, vector ()
        - R := distances, vector
    =#
    # unroll coefficients
    a = Θ[1:M]
    b = Θ[M+1:2*M]
    c = Θ[2*M+1:3*M+4]
    d = Θ[3*M+5:4*M+7]
    
    # b_i ≥ 0 for i > 1:
    t = b[2:M]
    bool = t .< 0.
    t[bool] = -t[bool]
    b[2:M] = t
    
    # d_i ≥ 0 for i > 0:
    bool = d .< 0.
    d[bool] = -d[bool]    
    
    # evaluate P:
    P = c[end-1]
    P = P .* ((R .- a[1]).^2 .+ (b[1] .* R))
    for i=2:M
        P .*= (R .- a[i]).^2 .+ (b[i]*R)
    end

    
    # eval Q:
    Q = (R .+ d[end]).*R
    for i=1:M+2
        Q .*= (R .- c[i]).^2 .+ d[i].*R
    end
    
    # eval potential:
    V = c[end] .+ (P ./ Q)
    return V
end

f_RMSE(X, Y) = √(sum((X .- Y) .^ 2)/length(X))

function f_least_squares(f_eval, Y, f_args...)
    Y_pred = f_eval(f_args...)
    res = sum((Y .- Y_pred).^2)
    return res
end

f_least_squares (generic function with 1 method)

In [212]:
M = 2
Θ = collect(LinRange(-1., -2., 21))
Θ = Θ[1:4*M+7] # 4M+7 params
println(Θ, size(Θ))
R = collect(LinRange(0., 5., 6)) .+ 1
println(R)
f_ratpot_2(Θ, R, M)

[-1.0, -1.05, -1.1, -1.15, -1.2000000000000002, -1.25, -1.2999999999999998, -1.35, -1.4, -1.4500000000000002, -1.5, -1.55, -1.6, -1.65, -1.7](15,)
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
[-1.1, 1.15]
[1.5, 1.55, 1.6, 1.65, 1.7]


6-element Vector{Float64}:
 -1.4538848497222299
 -1.4504032610414908
 -1.450089540167252
 -1.4500280979713995
 -1.450010836561844
 -1.4500048065726545

#### Optimize ratpot 2 !!

In [3]:
# prepare data:
H_data = readdlm("data/h2/h2_ground_w.txt")
R = H_data[:, 1]; V = H_data[:, 2]
Xs, Ys = shuffleobs((R, V))
train_data, test_data = splitobs((Xs, Ys); at=0.8)
R_train = copy(train_data[1]); V_train = copy(train_data[2]);
R_test = copy(test_data[1]); V_test = copy(test_data[2]);

# tuning parameters:
M = 20
ub = 10; lb = 0
θ = rand(4*M+7) .* (ub-lb) .+ lb # random between [a, b] = [-1, 1]
#println(f_ratpot_2(θ, R_train, M))
f_least_squares(f_ratpot_2, V_train, θ, R_train, M)

4404.864370899371

In [4]:
# optimize:

res = optimize(θ -> f_least_squares(f_ratpot_2, V_train, θ, R_train, M),
                θ, BFGS(),
                Optim.Options(iterations = 2000, show_trace=true); 
                autodiff = :forward
                )

Iter     Function value   Gradient norm 
     0     4.404864e+03     3.062211e+03
 * time: 0.01399993896484375
     1     2.793404e+03     2.440113e+03
 * time: 0.7990000247955322
     2     1.630070e+01     2.054986e+00
 * time: 0.812000036239624
     3     1.629872e+01     8.659113e-03
 * time: 0.8209998607635498
     4     1.623607e+01     5.600086e+00
 * time: 0.8889999389648438
     5     1.623432e+01     1.959398e+00
 * time: 0.9119999408721924
     6     1.621126e+01     8.370857e-01
 * time: 0.9229998588562012
     7     1.620387e+01     1.160661e+00
 * time: 0.9440000057220459
     8     1.619207e+01     1.135099e+00
 * time: 0.9639999866485596
     9     1.618045e+01     3.816732e-01
 * time: 0.9779999256134033
    10     1.613029e+01     1.295419e+01
 * time: 0.9939999580383301
    11     1.584807e+01     5.226360e+01
 * time: 1.0390000343322754
    12     1.584544e+01     5.503163e+01
 * time: 1.0829999446868896
    13     1.573687e+01     5.396900e+01
 * time: 1.0910000801

   120     1.033421e-04     3.278408e-02
 * time: 2.259000062942505
   121     9.759661e-05     2.962836e-02
 * time: 2.2690000534057617
   122     9.418553e-05     1.530917e-02
 * time: 2.2820000648498535
   123     9.204677e-05     3.411398e-03
 * time: 2.2960000038146973
   124     9.096486e-05     2.953289e-03
 * time: 2.302000045776367
   125     9.013070e-05     5.460474e-03
 * time: 2.309000015258789
   126     8.963581e-05     5.160004e-03
 * time: 2.315999984741211
   127     8.943762e-05     1.304418e-03
 * time: 2.3269999027252197
   128     8.911740e-05     3.887631e-03
 * time: 2.3329999446868896
   129     8.708551e-05     2.123379e-02
 * time: 2.3459999561309814
   130     8.373967e-05     2.494811e-02
 * time: 2.3559999465942383
   131     7.825803e-05     3.904916e-02
 * time: 2.365000009536743
   132     7.451538e-05     4.135836e-02
 * time: 2.372999906539917
   133     7.281997e-05     1.607791e-02
 * time: 2.381999969482422
   134     7.124462e-05     2.137468e-02


   240     8.393406e-06     7.189699e-04
 * time: 3.4129998683929443
   241     8.383559e-06     2.693552e-04
 * time: 3.421999931335449
   242     8.373837e-06     8.535068e-04
 * time: 3.430000066757202
   243     8.366244e-06     1.274197e-03
 * time: 3.437999963760376
   244     8.340059e-06     5.983128e-04
 * time: 3.4489998817443848
   245     8.306630e-06     1.754274e-03
 * time: 3.4619998931884766
   246     8.281918e-06     3.587015e-03
 * time: 3.4739999771118164
   247     8.254672e-06     3.657691e-03
 * time: 3.4789998531341553
   248     8.233224e-06     3.390320e-03
 * time: 3.484999895095825
   249     8.171514e-06     2.351461e-03
 * time: 3.493000030517578
   250     8.055552e-06     2.608718e-03
 * time: 3.500999927520752
   251     7.972921e-06     1.852941e-03
 * time: 3.51200008392334
   252     7.846761e-06     6.602139e-04
 * time: 3.5199999809265137
   253     7.793081e-06     3.921411e-03
 * time: 3.5320000648498535
   254     7.688746e-06     2.702694e-03
 

   361     3.878768e-06     1.247204e-03
 * time: 4.6519999504089355
   362     3.813689e-06     2.949838e-03
 * time: 4.662999868392944
   363     3.804321e-06     3.232468e-03
 * time: 4.675999879837036
   364     3.781788e-06     1.996771e-03
 * time: 4.683000087738037
   365     3.769767e-06     2.583854e-03
 * time: 4.691999912261963
   366     3.755177e-06     8.771728e-04
 * time: 4.694999933242798
   367     3.745423e-06     3.559733e-04
 * time: 4.70199990272522
   368     3.730694e-06     1.595028e-03
 * time: 4.710000038146973
   369     3.710211e-06     9.228669e-04
 * time: 4.7170000076293945
   370     3.698558e-06     1.005507e-03
 * time: 4.726999998092651
   371     3.686994e-06     7.179547e-04
 * time: 4.734999895095825
   372     3.675850e-06     6.846601e-04
 * time: 4.747999906539917
   373     3.663203e-06     2.265246e-03
 * time: 4.760999917984009
   374     3.652998e-06     1.846160e-03
 * time: 4.767999887466431
   375     3.630089e-06     1.022246e-03
 * tim

   482     1.444510e-06     1.550290e-03
 * time: 5.703999996185303
   483     1.439209e-06     1.001724e-03
 * time: 5.710999965667725
   484     1.434615e-06     7.841825e-04
 * time: 5.7209999561309814
   485     1.428665e-06     8.014224e-04
 * time: 5.728999853134155
   486     1.426292e-06     2.566635e-04
 * time: 5.740000009536743
   487     1.425648e-06     3.599164e-04
 * time: 5.746999979019165
   488     1.425435e-06     1.448209e-04
 * time: 5.756999969482422
   489     1.424800e-06     1.482361e-04
 * time: 5.766999959945679
   490     1.423052e-06     6.162086e-04
 * time: 5.772000074386597
   491     1.417523e-06     6.600706e-04
 * time: 5.7809998989105225
   492     1.391232e-06     1.721418e-03
 * time: 5.792999982833862
   493     1.382971e-06     1.405119e-03
 * time: 5.796000003814697
   494     1.367367e-06     5.009336e-04
 * time: 5.802000045776367
   495     1.351493e-06     9.023372e-04
 * time: 5.812000036239624
   496     1.329956e-06     1.659382e-03
 * ti

...

  1567     2.784578e-08     2.654445e-05
 * time: 14.477999925613403
  1568     2.782327e-08     7.257762e-05
 * time: 14.490999937057495
  1569     2.777407e-08     7.983030e-05
 * time: 14.49399995803833
  1570     2.776381e-08     7.072581e-05
 * time: 14.497999906539917
  1571     2.772000e-08     1.046459e-04
 * time: 14.520999908447266
  1572     2.767471e-08     1.187270e-04
 * time: 14.526999950408936
  1573     2.763844e-08     5.002302e-05
 * time: 14.52999997138977
  1574     2.760715e-08     8.637362e-05
 * time: 14.53600001335144
  1575     2.755464e-08     2.045461e-04
 * time: 14.54099988937378
  1576     2.740473e-08     2.875749e-04
 * time: 14.549000024795532
  1577     2.730739e-08     3.299978e-04
 * time: 14.552000045776367
  1578     2.713868e-08     1.438670e-04
 * time: 14.558000087738037
  1579     2.687864e-08     1.127424e-04
 * time: 14.562999963760376
  1580     2.659307e-08     2.083656e-04
 * time: 14.57800006866455
  1581     2.649832e-08     2.443378e-0

  1687     1.981331e-08     8.469641e-05
 * time: 15.389999866485596
  1688     1.980466e-08     1.405067e-05
 * time: 15.398000001907349
  1689     1.979199e-08     5.296152e-05
 * time: 15.40499997138977
  1690     1.978145e-08     6.558709e-05
 * time: 15.41599988937378
  1691     1.976345e-08     6.376730e-05
 * time: 15.420000076293945
  1692     1.975420e-08     3.834053e-05
 * time: 15.430000066757202
  1693     1.973670e-08     7.596640e-05
 * time: 15.437000036239624
  1694     1.971636e-08     7.929444e-06
 * time: 15.44700002670288
  1695     1.970755e-08     2.523706e-05
 * time: 15.450000047683716
  1696     1.969432e-08     6.375197e-05
 * time: 15.455999851226807
  1697     1.968753e-08     2.293858e-05
 * time: 15.460000038146973
  1698     1.968309e-08     6.615063e-05
 * time: 15.466000080108643
  1699     1.967459e-08     6.693854e-05
 * time: 15.470999956130981
  1700     1.966110e-08     2.352684e-05
 * time: 15.480000019073486
  1701     1.965461e-08     5.441345e

  1807     1.471043e-08     1.024274e-04
 * time: 16.277999877929688
  1808     1.465740e-08     1.799933e-04
 * time: 16.287999868392944
  1809     1.462866e-08     5.048260e-05
 * time: 16.2979998588562
  1810     1.459440e-08     1.190599e-04
 * time: 16.305000066757202
  1811     1.458071e-08     1.277944e-04
 * time: 16.308000087738037
  1812     1.455399e-08     1.497116e-04
 * time: 16.312000036239624
  1813     1.451666e-08     1.368081e-04
 * time: 16.32200002670288
  1814     1.448424e-08     1.456920e-04
 * time: 16.33299994468689
  1815     1.446093e-08     5.958031e-05
 * time: 16.335999965667725
  1816     1.443532e-08     1.817367e-05
 * time: 16.33899998664856
  1817     1.441787e-08     1.828646e-05
 * time: 16.346999883651733
  1818     1.439882e-08     5.534342e-05
 * time: 16.35599994659424
  1819     1.436990e-08     1.373292e-04
 * time: 16.36299991607666
  1820     1.434943e-08     1.606989e-04
 * time: 16.369999885559082
  1821     1.433930e-08     2.305833e-04


  1927     1.199080e-08     9.394632e-05
 * time: 17.149999856948853
  1928     1.197570e-08     1.713417e-05
 * time: 17.152999877929688
  1929     1.195674e-08     5.532750e-05
 * time: 17.16000008583069
  1930     1.195024e-08     7.004180e-05
 * time: 17.162999868392944
  1931     1.193227e-08     9.310323e-05
 * time: 17.168999910354614
  1932     1.190855e-08     9.979035e-05
 * time: 17.18400001525879
  1933     1.188438e-08     1.145837e-04
 * time: 17.194000005722046
  1934     1.185812e-08     2.248771e-05
 * time: 17.204999923706055
  1935     1.184291e-08     3.051145e-05
 * time: 17.2189998626709
  1936     1.181906e-08     1.032138e-04
 * time: 17.22499990463257
  1937     1.180969e-08     4.028805e-05
 * time: 17.227999925613403
  1938     1.179983e-08     1.040133e-04
 * time: 17.240000009536743
  1939     1.178199e-08     9.041857e-05
 * time: 17.250999927520752
  1940     1.173953e-08     2.705260e-05
 * time: 17.256999969482422
  1941     1.173241e-08     2.045151e-0

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     9.611939e-09

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 5.21e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.83e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.00e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.12e-03 ≰ 0.0e+00
    |g(x)|                 = 7.12e-05 ≰ 1.0e-08

 * Work counters
    Seconds run:   18  (vs limit Inf)
    Iterations:    2000
    f(x) calls:    5173
    ∇f(x) calls:   5173


In [5]:
# check RMSE:
V_pred = f_ratpot_2(res.minimizer, R_train, M)
println(f_RMSE(V_train, V_pred))
V_pred = f_ratpot_2(res.minimizer, R_test, M)
println(f_RMSE(V_test, V_pred))
for i=1:length(V_test)
    println(V_pred[i]," ",V_test[i])
end

4.234704519894572e-6
2.9276919358380464e-6
-0.0008473614097407878 -0.0008457269999999628
-0.07954291804404812 -0.07954594900000012
-0.1519622732506834 -0.15195909900000015
-0.11653569803675493 -0.1165328080000001
-0.03391427554983345 -0.03391220100000014
-0.005351304696011028 -0.005348754000000122
-0.16747835744279463 -0.16748270900000017
-0.17098887938290577 -0.17099233300000005
-5.478405504333317e-5 -5.306299999996433e-5
-0.031062824242911735 -0.031061373000000003
-0.010857211378577925 -0.010857708000000077
-0.026627807954423488 -0.026627746000000174
-0.15506383270077323 -0.15506615300000015
-0.07577424840002328 -0.07577727099999998
-8.232465193436234e-6 -6.208000000063052e-6
-0.0006030500384439641 -0.0006003549999999969
-0.0010596603561299767 -0.001059486000000165
-0.01959761557770706 -0.019599871000000046
-9.81964487805631e-5 -9.699600000012687e-5
-0.021836953193801134 -0.02183858500000002
-0.15661983418865094 -0.15662260100000003
-0.03696914403296936 -0.036966681
-0.16865910473891

### 8.1 Bonding features

In [75]:
function t_R_fun(R, R_up, R_low, e)
    R2 = R.^2
    return ((R2 .- R_low^2)./(R_up^2 .- R2)).^e
end

#function s_bond_strength

t_R_fun (generic function with 1 method)

In [76]:
R = [2.5, 3.1, 5, 1.1]
@benchmark t_R_fun(R, 6, 1, 2)

BenchmarkTools.Trial: 10000 samples with 982 evaluations.
 Range (min … max):  59.369 ns …  1.314 μs  ┊ GC (min … max): 0.00% … 92.58%
 Time  (median):     61.507 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   67.030 ns ± 56.162 ns  ┊ GC (mean ± σ):  5.19% ±  5.87%

  ▄██▄▅▄▃▂▁▁                                                  ▂
  ███████████▇▇▇▆▆▆▅▄▅▆▄▄▄▆▄▃▅▄▁▃▁▄▁▁▃▄▃▃▁▁▁▁▁▃▁▁▃▁▁▃▅▆▇▇██▇▇ █
  59.4 ns      Histogram: log(frequency) by time       119 ns <

 Memory estimate: 192 bytes, allocs estimate: 2.

### New bonding features (bump functions)

In [9]:
#=
these replaces the b array, z is equivalent to b_1 (d=1), while f is d ≥ 2
=#
# test the z function derivation:
A = [1 1 1 -1; 1 -1 -1 1; 0 -1 -5 3; 0 0 20 -6]
b = [1 0 0 0]'
r = A\b
print(r)
A*r

[0.5; 0.9375; 0.18750000000000006; 0.6250000000000001;;]

4×1 Matrix{Float64}:
 0.9999999999999999
 1.1102230246251565e-16
 2.220446049250313e-16
 0.0

In [10]:
function z_coord(t)
    t2 = t.^2
    z = 0.5 .- t.*(0.9375 .+ t2.*(0.18750000000000006.*t2 .- 0.6250000000000001))
    return z
end

c = [0.5; 0.9375; 0.18750000000000006; 0.6250000000000001]'
t = collect(LinRange(-1, 1, 10000))
println(z_coord(-1))
println(z_coord(1))
println(z_coord(t))

1.0
5.551115123125783e-17


[1.0, 0.9999999999899984, 0.99999999992, 0.9999999997300404, 0.9999999993601919, 0.9999999987505626, 0.9999999978412961, 0.9999999965725727, 0.9999999948846081, 0.9999999927176544, 0.9999999900119994, 0.9999999867079667, 0.9999999827459164, 0.9999999780662439, 0.9999999726093811, 0.9999999663157952, 0.9999999591259898, 0.9999999509805043, 0.9999999418199139, 0.9999999315848299, 0.9999999202158991, 0.9999999076538044, 0.9999998938392649, 0.9999998787130348, 0.9999998622159048, 0.9999998442887013, 0.9999998248722864, 0.9999998039075582, 0.9999997813354504, 0.999999757096933, 0.9999997311330111, 0.9999997033847263, 0.9999996737931554, 0.9999996422994113, 0.999999608844643, 0.9999995733700345, 0.9999995358168061, 0.9999994961262141, 0.99999945423955, 0.9999994100981412, 0.9999993636433508, 0.9999993148165782, 0.9999992635592576, 0.9999992098128596, 0.9999991535188902, 0.9999990946188915, 0.9999990330544408, 0.9999989687671511, 0.9999989016986716, 0.9999988317906867, 0.9999987589849164, 0.9

, 0.9906887986192845, 0.9906631612151832, 0.9906374797577225, 0.9906117542202014, 0.9905859845759468, 0.9905601707983144, 0.9905343128606885, 0.9905084107364821, 0.9904824643991363, 0.9904564738221213, 0.9904304389789355, 0.9904043598431057, 0.9903782363881878, 0.9903520685877656, 0.9903258564154516, 0.9902995998448871, 0.9902732988497414, 0.9902469534037126, 0.9902205634805273, 0.9901941290539404, 0.9901676500977354, 0.9901411265857243, 0.9901145584917472, 0.9900879457896732, 0.9900612884533992, 0.9900345864568513, 0.9900078397739833, 0.9899810483787777, 0.9899542122452454, 0.9899273313474257, 0.9899004056593864, 0.9898734351552237, 0.9898464198090619, 0.989819359595054, 0.9897922544873808, 0.9897651044602523, 0.9897379094879062, 0.989710669544609, 0.9896833846046548, 0.989656054642367, 0.9896286796320966, 0.9896012595482231, 0.9895737943651546, 0.989546284057327, 0.989518728599205, 0.9894911279652812, 0.9894634821300765, 0.9894357910681404, 0.9894080547540502, 0.989380273162412, 0.98

 0.9711311913633105, 0.9710799583912677, 0.9710286712343336, 0.9709773298795721, 0.9709259343140724, 0.9708744845249477, 0.9708229804993369, 0.9707714222244034, 0.9707198096873355, 0.970668142875346, 0.970616421775673, 0.9705646463755793, 0.9705128166623525, 0.9704609326233048, 0.9704089942457734, 0.9703570015171203, 0.9703049544247324, 0.9702528529560208, 0.970200697098422, 0.9701484868393973, 0.970096222166432, 0.9700439030670369, 0.9699915295287473, 0.9699391015391233, 0.9698866190857494, 0.9698340821562355, 0.9697814907382152, 0.9697288448193477, 0.9696761443873168, 0.9696233894298303, 0.9695705799346215, 0.9695177158894477, 0.9694647972820916, 0.96941182410036, 0.9693587963320842, 0.9693057139651209, 0.969252576987351, 0.9691993853866798, 0.9691461391510374, 0.9690928382683788, 0.9690394827266833, 0.9689860725139549, 0.9689326076182221, 0.9688790880275382, 0.968825513729981, 0.9687718847136527, 0.9687182009666803, 0.968664462477215, 0.9686106692334333, 0.9685568212235355, 0.968502

, 0.937271400551142, 0.9371910345113743, 0.9371106107522732, 0.9370301292727785, 0.9369495900718514, 0.9368689931484744, 0.9367883385016504, 0.9367076261304046, 0.9366268560337823, 0.9365460282108502, 0.9364651426606966, 0.93638419938243, 0.936303198375181, 0.9362221396381005, 0.9361410231703606, 0.9360598489711551, 0.9359786170396982, 0.9358973273752252, 0.935815979976993, 0.9357345748442788, 0.9356531119763815, 0.9355715913726204, 0.9354900130323365, 0.9354083769548913, 0.9353266831396676, 0.935244931586069, 0.9351631222935202, 0.9350812552614669, 0.9349993304893758, 0.9349173479767345, 0.9348353077230513, 0.9347532097278562, 0.9346710539906995, 0.9345888405111527, 0.9345065692888082, 0.9344242403232792, 0.9343418536142, 0.9342594091612259, 0.9341769069640327, 0.9340943470223174, 0.9340117293357981, 0.9339290539042133, 0.9338463207273229, 0.933763529804907, 0.9336806811367673, 0.933597774722726, 0.9335148105626259, 0.933431788656331, 0.9333487090037265, 0.9332655716047172, 0.93318237

...

, 0.0484345508500581, 0.04836522151674122, 0.04829594929339592, 0.0482267341747224, 0.04815757615539795, 0.04808847523007742, 0.04801943139339304, 0.047950444639954626, 0.047881514964349114, 0.047812642361141244, 0.04774382682487277, 0.04767506835006308, 0.04760636693120873, 0.047537722562783935, 0.04746913523924001, 0.04740060495500581, 0.04733213170448752, 0.04726371548206859, 0.04719535628210986, 0.04712705409894952, 0.04705880892690312, 0.046990620760263446, 0.04692248959330064, 0.046854415420262086, 0.04678639823537262, 0.04671843803283421, 0.04665053480682613, 0.04658268855150505, 0.046514899261004716, 0.046447166929436456, 0.046379491550888496, 0.04631187311942642, 0.04624431162909315, 0.04617680707390881, 0.046109359447870646, 0.0460419687449532, 0.04597463495910831, 0.04590735808426488, 0.04584013811432902, 0.045772975043184116, 0.04570586886469069, 0.045638819572686506, 0.04557182716098629, 0.045504891623382115, 0.04543801295364325, 0.045371191145515966, 0.04530442619272368, 

, 0.021616987894944095, 0.02157398126876936, 0.021531026500257677, 0.021488123572478868, 0.02144527246847694, 0.021402473171269698, 0.02135972566384914, 0.021317029929181275, 0.021274385950205865, 0.021231793709836855, 0.02118925319096171, 0.021146764376442362, 0.02110432724911465, 0.02106194179178772, 0.021019607987245226, 0.02097732581824463, 0.02093509526751719, 0.02089291631776813, 0.020850788951676413, 0.020808713151895253, 0.020766688901051267, 0.020724716181745373, 0.020682794976551955, 0.020640925268019528, 0.020599107038670406, 0.0205573402710007, 0.020515624947480438, 0.020473961050553158, 0.02043234856263687, 0.020390787466122606, 0.020349277743375582, 0.020307819376735092, 0.02026641234851384, 0.02022505664099833, 0.02018375223644875, 0.0201424991170997, 0.02010129726515869, 0.02006014666280742, 0.020019047292201386, 0.019977999135469715, 0.019937002174715168, 0.019896056392014305, 0.019855161769417595, 0.019814318288948918, 0.019773525932606062, 0.019732784682360394, 0.019

, 0.006753761817805126, 0.006732841620710861, 0.006711962303826868, 0.006691123837175628, 0.006670326190750309, 0.006649569334514882, 0.006628853238403509, 0.006608177872321319, 0.006587543206143631, 0.006566949209716566, 0.006546395852856934, 0.006525883105351793, 0.006505410936958722, 0.006484979317406325, 0.0064645882163929524, 0.006444237603588365, 0.006423927448631961, 0.0064036577211342705, 0.006383428390676016, 0.00636323942680872, 0.006343090799053874, 0.006322982476903993, 0.006302914429821671, 0.006282886627240025, 0.0062628990385628636, 0.006242951633164295, 0.006223044380388731, 0.006203177249551217, 0.006183350209937155, 0.006163563230802249, 0.006143816281372949, 0.006124109330845728, 0.006104442348387473, 0.006084815303135871, 0.006065228164198633, 0.0060456809006539936, 0.006026173481550268, 0.0060067058759064595, 0.005987278052711931, 0.005967889980926289, 0.005948541629479331, 0.005929232967271381, 0.005909963963173115, 0.005890734586025237, 0.0058715448046392504, 0.0

, 0.0009251448490328884, 0.0009193058278612054, 0.0009134908098677452, 0.000907699750888169, 0.0009019326067252753, 0.0008961893331491666, 0.0008904698858971938, 0.0008847742206740672, 0.0008791022931520232, 0.0008734540589699358, 0.0008678294737345937, 0.0008622284930197011, 0.0008566510723662657, 0.0008510971672824885, 0.0008455667332439298, 0.0008400597256933429, 0.0008345761000406737, 0.000829115811663006, 0.0008236788159047825, 0.0008182650680775283, 0.000812874523459961, 0.0008075071372980469, 0.0008021628648048895, 0.0007968416611608964, 0.0007915434815131683, 0.0007862682809768873, 0.0007810160146332068, 0.0007757866375313616, 0.000770580104687224, 0.0007653963710841927, 0.0007602353916721372, 0.0007550971213687863, 0.00074998151505834, 0.0007448885275925798, 0.0007398181137902027, 0.0007347702284368207, 0.0007297448262854611, 0.0007247418620558999, 0.0007197612904349393, 0.0007148030660769078, 0.0007098671436026049, 0.0007049534776003008, 0.0007000620226251253, 0.0006951927331

 2.739061888412664e-8, 2.1933756089165257e-8, 1.7254083606310644e-8, 1.3292033196332653e-8, 9.988000648775142e-9, 7.282345615422514e-9, 5.1153918878554805e-9, 3.4274273419399037e-9, 2.158703826804498e-9, 1.2494374423965837e-9, 6.398080953928797e-10, 2.699595547106526e-10, 8.000000661922968e-11, 1.0001555139638185e-11, 5.551115123125783e-17]


In [ ]:
t0 = [-1, 1.]
∂z = similar(t0)
ReverseDiff.gradient!(∂z, z_coord, t0)